In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()

In [ ]:
def step_B(query_imgs, train_img):
    
    train = cv2.imread(train_img,0)
    train2 = cv2.cvtColor(train,cv2.COLOR_GRAY2RGB)
    kp_train, des_train = sift.detectAndCompute(train,None)

    for query_img in query_imgs:
    
        query = cv2.imread(query_img,0)
        kp_query, des_query = sift.detectAndCompute(query,None)

        query_xc = np.mean(list(kp_query[i].pt[0] for i in range(len(kp_query))))
        query_yc = np.mean(list(kp_query[i].pt[1] for i in range(len(kp_query))))


        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 50)

        flann = cv2.FlannBasedMatcher(index_params, search_params)

        matches = flann.knnMatch(des_train,des_query,k=2)

        good_kp_query = []
        good_kp_train = []
        

        for m,n in matches:
            if m.distance < 0.9*n.distance:
                good_kp_query.append(kp_query[m.trainIdx])
                good_kp_train.append(kp_train[m.queryIdx])

        matches_4d = []

        for entry_query,entry_train in zip(good_kp_query,good_kp_train):

            v = ((query_xc-entry_query.pt[0]), (query_yc-entry_query.pt[1]))
            delta_angle = entry_train.angle - entry_query.angle
            ratio_scale = entry_train.size / entry_query.size
            train_xc = entry_train.pt[0] + ratio_scale * (np.cos(delta_angle) * v[0] - np.sin(delta_angle) * v[1])
            train_yc = entry_train.pt[1] + ratio_scale * (np.sin(delta_angle) * v[0] + np.cos(delta_angle) * v[1])

            matches_4d.append((train_xc,train_yc,delta_angle,ratio_scale))

        data_scale = list(matches_4d[i][3] for i in range(len(matches_4d)))
        counts_scale, bins_scale, patches_size = plt.hist(data_scale,bins='auto')
        img_scale = np.mean([bins_scale[np.argmax(counts_scale)],bins_scale[np.argmax(counts_scale)+1]])
        plt.close();

        data_angle = list(matches_4d[i][2] for i in range(len(matches_4d)))
        counts_angle, bins_angle, patches_angle = plt.hist(data_angle,bins='auto')
        plt.close();

        BIN_PRECISION_FACTOR = 0.33
        x_bin_size = img_scale * query.shape[1] * BIN_PRECISION_FACTOR
        y_bin_size = img_scale * query.shape[0] * BIN_PRECISION_FACTOR
        x_bins = int(np.ceil(train.shape[1]/x_bin_size)+2)
        y_bins = int(np.ceil(train.shape[0]/y_bin_size)+2)
        x_min = train.shape[1]/2 - x_bins/2 * x_bin_size
        y_min = train.shape[0]/2 - y_bins/2 * y_bin_size
        x_max = train.shape[1]/2 + x_bins/2 * x_bin_size
        y_max = train.shape[0]/2 + y_bins/2 * y_bin_size

        ANGLE_BINS = 7
        angle_bin_size = np.std(data_angle)/10
        angle_bin_center = np.mean(data_angle)
        angle_min = angle_bin_center - ANGLE_BINS/2 * angle_bin_size
        angle_max = angle_bin_center + ANGLE_BINS/2 * angle_bin_size

        #SCALE_BINS = 7
        scale_bin_size = np.std(data_scale)/10
        scale_bin_center = np.mean(data_scale)
        scale_min = 0 #scale_bin_center - SCALE_BINS/2 * scale_bin_size
        scale_max = scale_bin_center * 2 #scale_bin_center + SCALE_BINS/2 * scale_bin_size
        SCALE_BINS = int((scale_max-scale_min)/scale_bin_size)
        
        def votesOnMatch(m_4d):

            accumulator = np.zeros((x_bins,y_bins,ANGLE_BINS,SCALE_BINS))
            votes = {}

            for m in m_4d:
                try:
                    i = int(np.floor((m[0]-x_min)/x_bin_size))
                    j = int(np.floor((m[1]-y_min)/y_bin_size))
                    k = int(np.floor((m[2]-angle_min)/angle_bin_size))
                    l = int(np.floor((m[3]-scale_min)/scale_bin_size))
                    if i >= 0 and j >= 0 and k >= 0 and l >= 0:
                        accumulator[i][j][k][l]+=1
                        votes[(i,j,k,l)] = votes.get((i,j,k,l),[])
                        votes[(i,j,k,l)].append(m)
                except: pass
                try:
                    i = int(np.ceil((m[0]-x_min-x_bin_size/2)/x_bin_size))
                    j = int(np.ceil((m[1]-y_min-y_bin_size/2)/y_bin_size))
                    k = int(np.ceil((m[2]-angle_min-angle_bin_size/2)/angle_bin_size))
                    l = int(np.ceil((m[3]-scale_min-scale_bin_size/2)/scale_bin_size))
                    if i >= 0 and j >= 0 and k >= 0 and l >= 0:
                        accumulator[i][j][k][l]+=1
                        votes[(i,j,k,l)] = votes.get((i,j,k,l),[])
                        votes[(i,j,k,l)].append(m)
                except: pass

            return accumulator,votes

        accumulator,votes=votesOnMatch(matches_4d)

        NEIGH = 1
        T = 10+int(query.shape[0]/100)
        a = accumulator
        a1 = np.zeros((x_bins,y_bins,ANGLE_BINS,SCALE_BINS))
        for i in range(NEIGH,x_bins-NEIGH):
            for j in range(NEIGH,y_bins-NEIGH):
                for k in range(NEIGH,ANGLE_BINS-NEIGH):
                    for l in range(NEIGH,SCALE_BINS-NEIGH):
                        counter = 0
                        for x in range(-NEIGH,NEIGH+1):
                            for y in range(-NEIGH,NEIGH+1):
                                for z in range(-NEIGH,NEIGH+1):
                                    for w in range(-NEIGH,NEIGH+1):
                                        counter += a[i+x][j+y][k+z][l+w]
                                        if a1[i][j][k][l] == 0 and a[i][j][k][l] < a[i+x][j+y][k+z][l+w]:
                                            a1[i][j][k][l] = -1
                        if a1[i][j][k][l] == 0 and a[i][j][k][l] >= T:
                            a1[i][j][k][l] = counter
                        else:
                            a1[i][j][k][l] = 0
        a = a1
        a1 = np.zeros((x_bins,y_bins,ANGLE_BINS,SCALE_BINS))
        for i in range(NEIGH,x_bins-NEIGH):
            for j in range(NEIGH,y_bins-NEIGH):
                for k in range(NEIGH,ANGLE_BINS-NEIGH):
                    for l in range(NEIGH,SCALE_BINS-NEIGH):
                        counter = 0
                        for x in range(-NEIGH,NEIGH+1):
                            for y in range(-NEIGH,NEIGH+1):
                                for z in range(-NEIGH,NEIGH+1):
                                    for w in range(-NEIGH,NEIGH+1):
                                        counter += a[i+x][j+y][k+z][l+w]
                                        if a1[i][j][k][l] == 0 and a[i][j][k][l] < a[i+x][j+y][k+z][l+w]:
                                            a1[i][j][k][l] = -1
                        if a1[i][j][k][l] == 0 and a[i][j][k][l] >= T:
                            a1[i][j][k][l] = counter
                        else:
                            a1[i][j][k][l] = 0
        a = a1
        a1 = np.zeros((x_bins,y_bins,ANGLE_BINS,SCALE_BINS))
        for i in range(NEIGH,x_bins-NEIGH):
            for j in range(NEIGH,y_bins-NEIGH):
                for k in range(NEIGH,ANGLE_BINS-NEIGH):
                    for l in range(NEIGH,SCALE_BINS-NEIGH):
                        counter = 0
                        for x in range(-NEIGH,NEIGH+1):
                            for y in range(-NEIGH,NEIGH+1):
                                for z in range(-NEIGH,NEIGH+1):
                                    for w in range(-NEIGH,NEIGH+1):
                                        counter += a[i+x][j+y][k+z][l+w]
                                        if a1[i][j][k][l] == 0 and a[i][j][k][l] < a[i+x][j+y][k+z][l+w]:
                                            a1[i][j][k][l] = -1
                        if a1[i][j][k][l] == 0 and a[i][j][k][l] >= T:
                            a1[i][j][k][l] = counter
                        else:
                            a1[i][j][k][l] = 0

        threshold = 100+int(query.shape[0]/30)

        found_bins = np.argwhere(a1>=threshold)
        better_matches = []
        print(query_img,str(int(np.max(accumulator)))+'/'+str(T),'->',str(int(np.max(a1)))+'/'+str(threshold))

        for b in found_bins:
            matches = []
            for i in range(len(matches_4d)):
                if matches_4d[i] in votes[tuple(b)]:
                    matches.append((good_kp_query[i],good_kp_train[i]))
            better_matches.append(matches)

#        for m in matches_4d:
#            cv2.circle(train2, (int(m[0]),int(m[1])), radius=2, color=(255, 0, 0), thickness=-1)
        
        try:
            for matches in better_matches:
                src_pts = np.float32([ m[0].pt for m in matches ]).reshape(-1,1,2)
                dst_pts = np.float32([ m[1].pt for m in matches ]).reshape(-1,1,2)
                center = np.mean(dst_pts,axis=0)
                cv2.circle(train2, (int(center[0][0]),int(center[0][1])), radius=5, color=(0, 255, 0), thickness=10)
                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
                matchesMask = mask.ravel().tolist()
                h,w = query.shape
                pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                dst = cv2.perspectiveTransform(pts,M)
                draw_params = dict(matchColor = (0,0,0), # draw matches in green color
                               singlePointColor = None,
                               matchesMask = matchesMask, # draw only inliers
                               flags = 2)
                train2 = cv2.polylines(train2,[np.int32(dst)],True,(0,255,0),3, cv2.LINE_AA)
        except: pass
    plt.figure(figsize = (15,10))
    plt.imshow(train2),plt.show();

In [ ]:
query_imgs = ['models/0.jpg','models/1.jpg','models/11.jpg','models/19.jpg','models/24.jpg','models/25.jpg','models/26.jpg']
train_imgs = ['scenes/m1.png','scenes/m2.png','scenes/m3.png','scenes/m4.png','scenes/m5.png']
for train_img in train_imgs:
    step_B(query_imgs, train_img)

In [ ]:
## detect features in model 
#query = cv2.imread('models/25.jpg',0)
#kp_query, des_query = sift.detectAndCompute(query,None)
#
#query_xc = np.mean(list(kp_query[i].pt[0] for i in range(len(kp_query))))
#query_yc = np.mean(list(kp_query[i].pt[1] for i in range(len(kp_query))))

In [ ]:
##detect features in train
#train = cv2.imread('scenes/m5.png',0)
#kp_train, des_train = sift.detectAndCompute(train,None)

In [ ]:
##compute correspondences 
#
#FLANN_INDEX_KDTREE = 1
#index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
#search_params = dict(checks = 50)
#
#flann = cv2.FlannBasedMatcher(index_params, search_params)
#
#matches = flann.knnMatch(des_train,des_query,k=2)
#
#good_kp_query = []
#good_kp_train = []
#
#for m,n in matches:
#    if m.distance < 0.9*n.distance:
#        good_kp_query.append(kp_query[m.trainIdx])
#        good_kp_train.append(kp_train[m.queryIdx])

In [ ]:
#print(len(good_kp_query))
#print(len(set(good_kp_query)))

In [ ]:
#matches_4d = []
#
#for entry_query,entry_train in zip(good_kp_query,good_kp_train):
#
#    v = ((query_xc-entry_query.pt[0]), (query_yc-entry_query.pt[1]))
#    delta_angle = entry_train.angle - entry_query.angle
#    ratio_scale = entry_train.size / entry_query.size
#    train_xc = entry_train.pt[0] + ratio_scale * (np.cos(delta_angle) * v[0] - np.sin(delta_angle) * v[1])
#    train_yc = entry_train.pt[1] + ratio_scale * (np.sin(delta_angle) * v[0] + np.cos(delta_angle) * v[1])
#
#    matches_4d.append((train_xc,train_yc,delta_angle,ratio_scale))

In [ ]:
#data_scale = list(matches_4d[i][3] for i in range(len(matches_4d)))
#counts_scale, bins_scale, patches_size = plt.hist(data_scale,bins='auto')
#img_scale = np.mean([bins_scale[np.argmax(counts_scale)],bins_scale[np.argmax(counts_scale)+1]])
#plt.close();

In [ ]:
#data_angle = list(matches_4d[i][2] for i in range(len(matches_4d)))
#counts_angle, bins_angle, patches_angle = plt.hist(data_angle,bins='auto')
#plt.close();

In [ ]:
#BIN_PRECISION_FACTOR = 0.33
#x_bin_size = img_scale * query.shape[1] * BIN_PRECISION_FACTOR
#y_bin_size = img_scale * query.shape[0] * BIN_PRECISION_FACTOR
#x_bins = int(np.ceil(train.shape[1]/x_bin_size)+2)
#y_bins = int(np.ceil(train.shape[0]/y_bin_size)+2)
#x_min = train.shape[1]/2 - x_bins/2 * x_bin_size
#y_min = train.shape[0]/2 - y_bins/2 * y_bin_size
#x_max = train.shape[1]/2 + x_bins/2 * x_bin_size
#y_max = train.shape[0]/2 + y_bins/2 * y_bin_size
#print(x_bin_size,y_bin_size)
#print(x_bins,y_bins)
#print(x_min,y_min)
#print(x_max,y_max)

In [ ]:
#ANGLE_BINS = 7
#angle_bin_size = np.std(data_angle)/10
#angle_bin_center = np.mean(data_angle)
#angle_min = angle_bin_center - ANGLE_BINS/2 * angle_bin_size
#angle_max = angle_bin_center + ANGLE_BINS/2 * angle_bin_size
#print(angle_bin_size)
#print(angle_min,angle_max)

In [ ]:
##SCALE_BINS = 7
#scale_bin_size = np.std(data_scale)/10
#scale_bin_center = np.mean(data_scale)
#scale_min = 0 #scale_bin_center - SCALE_BINS/2 * scale_bin_size
#scale_max = scale_bin_center * 2 #scale_bin_center + SCALE_BINS/2 * scale_bin_size
#SCALE_BINS = int((scale_max-scale_min)/scale_bin_size)
#print(SCALE_BINS)
#print(scale_bin_size)
#print(scale_min,scale_max)

In [ ]:
#def votesOnMatch(m_4d):
#
#    accumulator = np.zeros((x_bins,y_bins,ANGLE_BINS,SCALE_BINS))
#    votes = {}
#
#    for m in m_4d:
#        try:
#            i = int(np.floor((m[0]-x_min)/x_bin_size))
#            j = int(np.floor((m[1]-y_min)/y_bin_size))
#            k = int(np.floor((m[2]-angle_min)/angle_bin_size))
#            l = int(np.floor((m[3]-scale_min)/scale_bin_size))
#            if i >= 0 and j >= 0 and k >= 0 and l >= 0:
#                accumulator[i][j][k][l]+=1
#                votes[(i,j,k,l)] = votes.get((i,j,k,l),[])
#                votes[(i,j,k,l)].append(m)
#        except: pass
#        try:
#            i = int(np.ceil((m[0]-x_min-x_bin_size/2)/x_bin_size))
#            j = int(np.ceil((m[1]-y_min-y_bin_size/2)/y_bin_size))
#            k = int(np.ceil((m[2]-angle_min-angle_bin_size/2)/angle_bin_size))
#            l = int(np.ceil((m[3]-scale_min-scale_bin_size/2)/scale_bin_size))
#            if i >= 0 and j >= 0 and k >= 0 and l >= 0:
#                accumulator[i][j][k][l]+=1
#                votes[(i,j,k,l)] = votes.get((i,j,k,l),[])
#                votes[(i,j,k,l)].append(m)
#        except: pass
#        
#    return accumulator,votes

In [ ]:
#accumulator,votes=votesOnMatch(matches_4d)

In [ ]:
#print(sorted(np.ravel(accumulator))[::-1])

In [ ]:
#NEIGH = 1
#T = 10
#a = accumulator
#a1 = np.zeros((x_bins,y_bins,ANGLE_BINS,SCALE_BINS))
#for i in range(NEIGH,x_bins-NEIGH):
#    for j in range(NEIGH,y_bins-NEIGH):
#        for k in range(NEIGH,ANGLE_BINS-NEIGH):
#            for l in range(NEIGH,SCALE_BINS-NEIGH):
#                counter = 0
#                for x in range(-NEIGH,NEIGH+1):
#                    for y in range(-NEIGH,NEIGH+1):
#                        for z in range(-NEIGH,NEIGH+1):
#                            for w in range(-NEIGH,NEIGH+1):
#                                counter += a[i+x][j+y][k+z][l+w]
#                                if a1[i][j][k][l] == 0 and a[i][j][k][l] < a[i+x][j+y][k+z][l+w]:
#                                    a1[i][j][k][l] = -1
#                if a1[i][j][k][l] == 0 and a[i][j][k][l] > T:
#                    a1[i][j][k][l] = counter
#                else:
#                    a1[i][j][k][l] = 0
#a = a1
#a1 = np.zeros((x_bins,y_bins,ANGLE_BINS,SCALE_BINS))
#for i in range(NEIGH,x_bins-NEIGH):
#    for j in range(NEIGH,y_bins-NEIGH):
#        for k in range(NEIGH,ANGLE_BINS-NEIGH):
#            for l in range(NEIGH,SCALE_BINS-NEIGH):
#                counter = 0
#                for x in range(-NEIGH,NEIGH+1):
#                    for y in range(-NEIGH,NEIGH+1):
#                        for z in range(-NEIGH,NEIGH+1):
#                            for w in range(-NEIGH,NEIGH+1):
#                                counter += a[i+x][j+y][k+z][l+w]
#                                if a1[i][j][k][l] == 0 and a[i][j][k][l] < a[i+x][j+y][k+z][l+w]:
#                                    a1[i][j][k][l] = -1
#                if a1[i][j][k][l] == 0 and a[i][j][k][l] > T:
#                    a1[i][j][k][l] = counter
#                else:
#                    a1[i][j][k][l] = 0

In [ ]:
#print(sorted(np.ravel(a1))[::-1])

In [ ]:
#ABS_THRESH = 30
#REL_THRESH = 0.5
#MULTI = 2
#threshold = 0
#a2 = sorted(np.ravel(a1))[::-1]
#if a2[0] >= ABS_THRESH:
#    threshold = a2[0]
#    for i in range(1,len(a2)-1):
#        if a2[i]-a2[i+1] > MULTI*(a2[0]-a2[i]):
#            threshold = a2[i]
#            break
#print(threshold)

In [ ]:
#threshold = 150

In [ ]:
#print(votes)

In [ ]:
#found_bins = np.argwhere(a1>=threshold)
#better_matches = []
#
#for b in found_bins:
#    matches = []
#    for i in range(len(matches_4d)):
#        if matches_4d[i] in votes[tuple(b)]:
#            matches.append((good_kp_query[i],good_kp_train[i]))
#    better_matches.append(matches)

In [ ]:
#train2 = cv2.cvtColor(train,cv2.COLOR_GRAY2RGB)
#
#for m in matches_4d:
#    cv2.circle(train2, (int(m[0]),int(m[1])), radius=2, color=(255, 0, 0), thickness=-1)
#
#for matches in better_matches:
#    src_pts = np.float32([ m[0].pt for m in matches ]).reshape(-1,1,2)
#    dst_pts = np.float32([ m[1].pt for m in matches ]).reshape(-1,1,2)
#    center = np.mean(dst_pts,axis=0)
#    cv2.circle(train2, (int(center[0][0]),int(center[0][1])), radius=5, color=(0, 255, 0), thickness=10)
#    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
#    matchesMask = mask.ravel().tolist()
#    h,w = query.shape
#    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
#    dst = cv2.perspectiveTransform(pts,M)
#    draw_params = dict(matchColor = (0,0,0), # draw matches in green color
#                   singlePointColor = None,
#                   matchesMask = matchesMask, # draw only inliers
#                   flags = 2)
#    train2 = cv2.polylines(train2,[np.int32(dst)],True,(0,255,0),3, cv2.LINE_AA)
#plt.figure(figsize = (15,10))
#plt.imshow(train2),plt.show();